<a href="https://colab.research.google.com/github/GoeSoft/-Django/blob/main/%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B1%D0%B0%D0%B7_%D0%B7%D0%BD%D0%B0%D0%BD%D0%B8%D0%B9_%D0%90%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D0%B0_%D0%B1%D0%B0%D0%B7%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q faiss-cpu==1.10.0 openai==1.70.0 tiktoken==0.9.0 langchain_openai==0.3.12 langchain_community==0.3.21

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 85.0 MB/s eta 0:00:00


In [3]:
# запустите эту ячейку, если используете секретный ключ в колабе

from openai import OpenAI
import os
from google.colab import userdata
# Получение API ключа из пользовательских данных Colab и установка его как переменной среды
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI()

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document

In [5]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [6]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1WMp-uMA4_59Ybp2ynCvrFIPV1OwGHQA4zvW526BawJM/edit?tab=t.0#heading=h.u417kcfvuqpv') # заполните аргумент функции

In [7]:
data_from_url[:2000]

'\ufeffАдминистративный регламент\r\nФедеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня\r\nI. Общие положения\r\n*Предмет регулирования регламента*\r\n1. Административный регламент Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - Регламент) определяет сроки и последовательность административных процедур (действий) при предоставлении Федеральной службой по надзору в сфере природопользования (далее - Росприроднадзор) и ее территориальными органами государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - государственная услуга), а также порядок взаимодействия между структурными подразделениями Росприроднадзора и территориальными органами Росприроднадзора

In [8]:
def text_to_markdown(text):
    # Добавляем заголовок 1 уровня на основе римских чисел (без переноса строки)
    # и дублируем его строчкой ниже - иначе эта информация перенесется в метаданные, а порой она бывает полезной.
    def replace_header1(match):
        return f"# {match.group(2)}\n{match.group(2)}"

    text = re.sub(r'^(I{1,3}|IV|V)\. (.+)', replace_header1, text, flags=re.M)

    # Добавляем текст, выделенный жирным шрифтом (он заключен между *)
    # и дублируем его строчкой ниже
    def replace_header2(match):
        return f"## {match.group(1)}\n{match.group(1)}"

    text = re.sub(r'\*([^\*]+)\*', replace_header2, text)

    return text

In [9]:
markdown = text_to_markdown(data_from_url)
print(markdown[:15000])

﻿Административный регламент
Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня
# Общие положения
Общие положения
## Предмет регулирования регламента
Предмет регулирования регламента
1. Административный регламент Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - Регламент) определяет сроки и последовательность административных процедур (действий) при предоставлении Федеральной службой по надзору в сфере природопользования (далее - Росприроднадзор) и ее территориальными органами государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - государственная услуга), а также порядок взаимодействия между структурными подразделениями Росприроднадзора и территориал

In [10]:
# Инструкция в system
system = load_document_text('https://docs.google.com/document/d/1Eeio_oWIQHwfNWR_enEVhTJbQGS0qhvT-18mWwSJILc/edit?tab=t.0')

In [11]:
print(system[:1000])

﻿Вы — официальный нейро-ассистент по вопросам государственной экологической экспертизы. 
Ваша задача — предоставлять точную, структурированную информацию исключительно на основе утверждённого регламента проведения государственной экологической экспертизы.


 Правила поведения:
1. Отвечайте ТОЛЬКО на основе предоставленного регламента. Не добавляйте личных интерпретаций, примеров или внешней информации.
2. Если вопрос выходит за рамки регламента (например, касается судебной практики, бытовых советов, IT, политики, медицины и т.п.), вежливо сообщите:  
   > "Я могу отвечать только на вопросы, связанные с регламентом государственной экологической экспертизы. Ваш запрос находится вне этой темы."
3. Если информация по вопросу отсутствует в документе, скажите:  
   > "Согласно доступному регламенту, информация по данному вопросу отсутствует."
4. При наличии данных — структурируйте ответ:  
   - Краткий ответ.  
   - Ссылка на раздел/статью регламента (если указана).  
   - Переч


In [12]:
# делим БЗ на чанки при помощи MarkdownHeaderTextSplitter, так как предварительно мы ее разметили именно таким образом
headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2")
    ]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
fragments = markdown_splitter.split_text(markdown)

In [13]:
# выводим несколько чанков, чтобы убедиться, что все получилось в необходимом нам формате
fragments[2:4]

[Document(metadata={'Header 1': 'Общие положения', 'Header 2': 'Предмет регулирования регламента'}, page_content='Предмет регулирования регламента\n1. Административный регламент Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - Регламент) определяет сроки и последовательность административных процедур (действий) при предоставлении Федеральной службой по надзору в сфере природопользования (далее - Росприроднадзор) и ее территориальными органами государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - государственная услуга), а также порядок взаимодействия между структурными подразделениями Росприроднадзора и территориальными органами Росприроднадзора, их должностными лицами и заявителями при предоставлении государственной услуги.'),
 Document(metadata={'Header 1': 'Общие положения', 

In [14]:
len(fragments)

41

In [15]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(fragments, embeddings)

In [17]:
def answer_index(system, topic, search_index, verbose=0):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос пользователя: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-4o-mini",             # выберите модель самостоятельно
        messages=messages,
        temperature= 0.5          # укажите значение
    )
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

In [19]:
topic= "какой срок проведения государственной экологической экспертизы проектной документации объектов?"                                         # Напишите сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

'Здравствуйте! Я — ваш консультант по регламенту государственной экологической экспертизы. Задайте ваш вопрос.\n\nСогласно предоставленной информации, срок проведения государственной экологической экспертизы проектной документации объектов инфраструктуры территорий опережающего социально-экономического развития не может превышать 45 календарных дней после ее предварительной оплаты в полном объеме.\n\nДля остальных объектов срок проведения государственной экологической экспертизы не должен превышать 2 месяцев с начала ее проведения, определенной в соответствии с датой приказа или решения об организации и проведении государственной экологической экспертизы.\n\n- Ссылка на раздел/статью регламента: Отрывок документа №2, пункты 10 и 11.\n- Перечень сроков:\n  - 45 календарных дней — для объектов инфраструктуры территорий опережающего социально-экономического развития.\n  - 2 месяца — для остальных объектов.'

In [ ]:
topic= ""                                         # Напишите сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

In [ ]:
topic= ""                                         # Напишите сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans